# Bollinger Squeeze Breakout — 2800.HK

- Logic: identify low-volatility ``squeeze'' regimes (bandwidth in low quantile), then trade breakouts.
- This is a more advanced Bollinger variant than simple mean reversion.
- Backtest: close-to-close; transaction cost charged on position changes (bps).

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from fyp_trading.data import fetch_prices
from fyp_trading.backtest import equity_and_stats_from_positions
from fyp_trading.strategies import BollingerSqueezeConfig, bollinger_squeeze_breakout_signal

TICKER = "2800.HK"
PERIOD = "3y"
INTERVAL = "1d"
TRANSACTION_COST_BP = 2.0
ALLOW_SHORT = False

CFG = BollingerSqueezeConfig(period=20, num_std=2.0, squeeze_lookback=120, squeeze_quantile=0.2, allow_short=ALLOW_SHORT)

df = fetch_prices(TICKER, PERIOD, INTERVAL)
close = df["Close"].copy()

pos = bollinger_squeeze_breakout_signal(close, CFG)

next_ret = np.log(close.shift(-1) / close)
simple_ret = (np.exp(next_ret) - 1.0).dropna()

bt_df, stats = equity_and_stats_from_positions(
    simple_return=simple_ret,
    position=pos.loc[simple_ret.index],
    transaction_cost_bp=TRANSACTION_COST_BP,
)

print(pd.Series(stats))

plt.figure(figsize=(12,4))
plt.plot(bt_df["strategy_equity"], label="Bollinger Squeeze breakout")
plt.plot(bt_df["buyhold_equity"], label="Buy&Hold", linestyle="--")
plt.title(
    f"Bollinger Squeeze Breakout | {TICKER} | period={CFG.period}, q={CFG.squeeze_quantile}, cost={TRANSACTION_COST_BP:.0f}bps"
)
plt.legend()
plt.grid(alpha=0.3)
plt.show()
